In [3]:
import os
import csv
import json
import pickle
import requests
from time import sleep

Consumer Key	XZoHfEhHTquhRinLcNrS
Consumer Secret	QozVgMYFgZkYwcCdXNOhApHklUlVKUxe
Request Token URL	https://api.discogs.com/oauth/request_token
Authorize URL	https://www.discogs.com/oauth/authorize
Access Token URL	https://api.discogs.com/oauth/access_token

In [4]:
key = 'XZoHfEhHTquhRinLcNrS'
secret = 'QozVgMYFgZkYwcCdXNOhApHklUlVKUxe'

### Get release thumbs by artist (much faster)

In [28]:
releases = set([_[6:-4] for _ in os.listdir('thumbs')])

In [37]:
artist_id = 151642 #1

In [41]:
while True:
    print(len(releases), end='\r')
    while True:
        sleep(1)
        url = f'https://api.discogs.com/artists/{artist_id}/releases?key={key}&secret={secret}'
        r = requests.get(url, headers={'User-agent': 'FooBarApp/3.0'})
        if r.status_code == 200 or r.status_code == 404:
            break
    response = json.loads(r.text)
    if 'message' in response and response['message'] == 'Artist not found.':
        artist_id += 1
        continue

    for release in response.get('releases', []):
        image_url = release.get('thumb', '')

        if 'id' in release and image_url != '' and str(release['id']) not in releases:
            while True:
                r = requests.get(image_url, headers={'User-agent': 'FooBarApp/3.0'})
                if r.status_code == 200 or r.status_code == 404:
                    break
            if r.status_code == 200:
                with open(f'thumbs/thumb_{release["id"]}.jpg', 'wb') as file:
                    file.write(r.content)
            releases.add(str(release['id']))
                
    artist_id += 1

KeyboardInterrupt: 

In [40]:
artist_id

153329

### Need to differentiate between master and release (record artist and title while you are at it)

In [39]:
artist_id = 88
url = f'https://api.discogs.com/artists/{artist_id}/releases?key={key}&secret={secret}'
r = requests.get(url, headers={'User-agent': 'FooBarApp/3.0'})
response = json.loads(r.text)
for release in response['releases']:
    id = release['main_release'] if release['type'] == 'master' else release['id']
    print(id, release.get('thumb', ''))

1147370 https://img.discogs.com/3Wi0j8Q9_kfD3ZhVXvBL4fXYsLE=/fit-in/150x150/filters:strip_icc():format(jpeg):mode_rgb():quality(40)/discogs-images/R-1147370-1589694143-9825.jpeg.jpg
27886 https://img.discogs.com/UinUWA3QLGs_Zn5-WHQ-j6Y5KzI=/fit-in/150x150/filters:strip_icc():format(jpeg):mode_rgb():quality(40)/discogs-images/R-27886-1556471294-9192.png.jpg
18042 https://img.discogs.com/5Po2wSrAkJsR9Bnhig0cVhUHL0c=/fit-in/150x150/filters:strip_icc():format(jpeg):mode_rgb():quality(40)/discogs-images/R-18042-1155811003.jpeg.jpg
71066 https://img.discogs.com/eNUi761BuEbved08x_bvFkHsxFc=/fit-in/150x150/filters:strip_icc():format(jpeg):mode_rgb():quality(40)/discogs-images/R-71066-1288393531.jpeg.jpg
463306 https://img.discogs.com/t7UNL2Ce5G0pAVgbrM3G29XBENA=/fit-in/150x150/filters:strip_icc():format(jpeg):mode_rgb():quality(40)/discogs-images/R-463306-1117154918.jpg.jpg
5182 https://img.discogs.com/b1vovoeV4F20tHZCf5ZBrh5o8Nw=/fit-in/150x150/filters:strip_icc():format(jpeg):mode_rgb():qual